In [ ]:
import numpy as np
import pandas as pd

In [6]:
def free_part(kh):
    res = {}
    # Not the optimal way; while also not the bottleneck for computation
    # The size of kh is not big
    # TODO: improve this function
    for key1 in kh.keys():
        for key2 in kh[key1].keys():
            a = kh[key1][key2]
            gens = a.gens()
            n = len([gen for gen in gens if gen.additive_order() == +Infinity])
            if n>0:
                res[(key1, key2)] = n 
    return res

In [7]:
def torsion_part(kh): # key - orders of the generator; value - number of generator with that order
    res = {}
    # Not the optimal way; while also not the bottleneck for computation
    # The size of kh is not big
    # TODO: improve this function
    for key1 in kh.keys():
        for key2 in kh[key1].keys():
            val = {}
            a = kh[key1][key2]
            gens = a.gens()
            for gen in gens:
                b = gen.additive_order()
                if b == +Infinity:
                    continue
                elif b in val.keys():
                    val[b] += 1
                else:
                    val[b] = 1   
            if len(list(val.keys()))>0:
                res[(key1, key2)] = val
    return res

In [13]:
import sys
sys.path.append('../')

from scripts.dataGeneration import getRandomWord, count_FP, count_TP

S = [-3, -2, -1, 1, 2, 3]

B = BraidGroup(4)

In [9]:
def generate_dataset(n,m):

# n is the desired size of the dataset
# m determines how long the words used to generate the links are going to be
# WARNING: m >= 9 will make the khovanov homology very long so be careful.
# I am personally using m = 8
    
    data = {
        'braid': [],
        'khovanov_homology': []
        'free_part': [],
        'torsion_part': [],
        'free_part_count': [],
        'torsion_part_count': []
    }

    for i in range(n):
        W = getRandomWord(m, S)
        kh = Link(B(W)).khovanov_homology()
        
        if kh in data["khovanov_homology"]:
            continue
        data["braid"].append(W)
        data["khovanov_homology"].append(kh)
        data["free_part"].append(free_part(kh))
        data["torsion_part"].append(torsion_part(kh))
        data["free_part_count"].append(count_FP(kh))
        data["torsion_part_count"].append(count_TP(kh))

    df = pd.DataFrame(data)
    
    df.to_csv(f'{n}_{m}_dataset.csv')
